In [3]:
import os, sys
import json
import time, datetime
import pandas as pd


import Levenshtein


In [2]:
import torch

In [4]:
maven_path = r'D:\Lab\Vulnerabilities library\FastChat\VuiLibGen\data\maven_corpus_new.json'

In [5]:
with open(maven_path, 'r') as f:
    maven_corpus = json.load(f)

In [ ]:
maven_corpus

In [20]:
#Ground truth of library name 
lib_names = set([lib['name'] for lib in maven_corpus])

In [21]:
lib_names

{'io.github.cdimascio:essence',
 'org.openehealth.ipf.platform-camel:ipf-platform-camel-ihe-atna',
 'com.typesafe.akka:akka-kernel_2.10.0-RC2',
 'love.forte.simple-robot:spring-boot-starter',
 'cn.zhangsw:mysql2mybatis-generator-ui',
 'org.netbeans.modules:org-netbeans-modules-spring-webmvc',
 'org.goots.hiderdoclet:doclet',
 'jp.co.bizreach:dynamodb4s_2.11',
 'org.picketlink:picketlink-social-webapps',
 'org.activecomponents.jadex:jadex-applications-micro',
 'com.typesafe:ssl-config-akka_2.12.0-M4',
 'com.github.adedayo.eclipse.sdk:org.eclipse.jdt.launching',
 'com.vividsolutions:jts-example',
 'me.masahito:yet_another_ltsv_scala_2.9.2',
 'com.joom.colonist:colonist-processor',
 'one.irradia.http:one.irradia.http.vanilla',
 'com.wootric:analytics-integration-wootric',
 'ch.datascience:renga-graph-typesystem-implementation_2.11',
 'org.prompto:SOLRStore',
 'com.numdata:numdata-velocity',
 'com.netflix.denominator:denominator-dynect',
 'su.lychee:android-bindings',
 'uk.nhs.ciao:ciao-sp

In [ ]:
artifacts = {item.split(':')[-1]: set() for item in lib_names}
for item in lib_names:
    components = item.split(':')
    artifacts[components[-1]].add(item)

In [ ]:
type(artifacts)

In [ ]:
artifacts

In [ ]:
weights = (1, 2, 2)

# Search the cloest suffix in the dict with input suffix
def cloest_artifact(artifact_id):
    global artifacts, weights
    if artifact_id in artifacts:
        return artifact_id

    distances = [(Levenshtein.distance(artifact_id, item,\
                    weights = weights), item) for item in artifacts]
    return min(distances)[1]

# Search for the cloest prefix in the group of all corresponding prefix with the known suffix
def cloest_group(group_id, groups):
    if len(groups) == 0:
        return group_id
    if len(groups) == 1:
        return next(iter(groups))
    
    global weights
    distances = [(Levenshtein.distance(group_id, item.split(':')[-2],\
                    weights = weights), item) for item in groups]
    return min(distances)[1]


# Local search Algorithm
def closest_lib(label):
    global lib_names
    if label in lib_names:
        return label
    if len(label.split(':')) > 1:
        group_id, artifact_id = label.split(':')[-2], label.split(':')[-1]
    else:
        group_id, artifact_id = "", label.split(':')[-1]
    if artifact_id in artifacts:
        return cloest_group(group_id, artifacts[artifact_id])
    else:
        advanced_artifact_id = cloest_artifact(artifact_id)
        return cloest_group(group_id, artifacts[advanced_artifact_id])


In [24]:
from post import closest_lib

In [16]:
label = 'org.apache.flume:flume-pdsadasdasdad'

In [19]:
closest_lib(label)

'io.fluidsonic.pdf:fluid-pdf-metadata'